In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sps 

combine the two scores

In [2]:
zscore = pd.read_csv('zscore.csv')

In [3]:
autoscore = pd.read_csv('autoscore.csv')

In [4]:
scores = pd.merge(zscore,autoscore,left_on='RECORD',right_on='index')

Add bin/rank columns

In [5]:
del scores['Unnamed: 0_x']
del scores['Unnamed: 0_y']
del scores['index']
del scores['Score2']

In [6]:
temp = scores['Score1'].sort_values().reset_index()
temp['RECORD'] = temp['index']+1
del temp['index']

In [7]:
temp = temp.reset_index()
temp['manhattan_bin_score'] = temp['index']+1
del temp['index']

In [8]:
scores1 = pd.merge(scores,temp,left_on='RECORD',right_on='RECORD')
del scores1['Score1_y']

In [9]:
scores1.columns = (['RECORD','Score1','score_auto','manhattan_bin_score'])

In [10]:
scores1.sort_values('manhattan_bin_score').head()

,RECORD,Score1,score_auto,manhattan_bin_score
185222,185223,0.159432,0.807662,1
200444,200445,0.166762,1.640552,2
184114,184115,0.166811,2.755366,3
184965,184966,0.174572,3.296350,4
878655,878656,0.176785,1.593520,5


In [11]:
temp = scores['score_auto'].sort_values().reset_index()
temp['RECORD'] = temp['index']+1
del temp['index']
temp = temp.reset_index()
temp['auto_bin_score'] = temp['index']+1
del temp['index']


In [12]:
final = pd.merge(scores1,temp,left_on='RECORD',right_on='RECORD')
del final['score_auto_y']
final.columns = (['RECORD','Score1','score_auto','manhattan_bin_score','auto_bin_score'])

In [13]:
final.head()

,RECORD,Score1,score_auto,manhattan_bin_score,auto_bin_score
0,1,20.893195,21.615629,1069363,1069420
1,2,31.238281,31.218499,1070096,1070079
2,3,21.162240,21.267942,1069397,1069367
3,4,17.416420,16.688151,1068747,1068481
4,5,83.847093,83.136734,1070704,1070701


Add weighted score


In [14]:
final['weightedscore']=0.5*final['manhattan_bin_score']+0.5*final['auto_bin_score']

In [15]:
final.sort_values('weightedscore').head()


,RECORD,Score1,score_auto,manhattan_bin_score,auto_bin_score,weightedscore
714599,714600,0.701197,0.093585,17628,6693,12160.5
383627,383628,0.640804,0.115492,12622,14522,13572.0
362833,362834,0.771293,0.066223,27921,1439,14680.0
376289,376290,0.808918,0.062823,32141,1094,16617.5
275294,275295,0.653222,0.128400,13426,20306,16866.0


In [16]:
final.sort_values('auto_bin_score').head()

,RECORD,Score1,score_auto,manhattan_bin_score,auto_bin_score,weightedscore
449924,449925,1.196094,0.020411,473262,1,236631.5
398714,398715,1.114693,0.021359,295233,2,147617.5
833631,833632,1.078294,0.021545,208740,3,104371.5
242016,242017,1.048296,0.023378,141141,4,70572.5
675614,675615,1.144695,0.023743,362100,5,181052.5


In [17]:
final.sort_values('manhattan_bin_score').head()

,RECORD,Score1,score_auto,manhattan_bin_score,auto_bin_score,weightedscore
185222,185223,0.159432,0.807662,1,373725,186863.0
200444,200445,0.166762,1.640552,2,512885,256443.5
184114,184115,0.166811,2.755366,3,666049,333026.0
184965,184966,0.174572,3.296350,4,717616,358810.0
878655,878656,0.176785,1.593520,5,507790,253897.5


merge the weightedscore with the filled dataset

In [18]:
temp = final[['RECORD','manhattan_bin_score','auto_bin_score','weightedscore']]

In [19]:
data = pd.read_csv('NY property data_FILLED_update.csv')

In [20]:
del data['Unnamed: 0']
del data['Unnamed: 0.1']
del data['Unnamed: 0.1.1']

In [21]:
mergeddata = pd.merge(data,temp,left_on='RECORD',right_on='RECORD')

In [22]:
mergeddata.head()

,RECORD,BBLE,B,BLOCK,LOT,EASEMENT,OWNER,BLDGCL,TAXCLASS,LTFRONT,...,meanFULLVAL,meanAVLAND,meanAVTOT,medianLTFRONT,medianLTDEPTH,medianBLDFRONT,medianBLDDEPTH,manhattan_bin_score,auto_bin_score,weightedscore
0,1,1000010101,1,1,101,NaN,U S GOVT LAND & BLDGS,P7,4,500.0,...,6859012.0,1228817.0,3062656.0,223.0,53.0,132.0,75.0,1069363,1069420,1069391.5
1,2,1000010201,1,1,201,NaN,U S GOVT LAND & BLDGS,Z9,4,27.0,...,6859012.0,1228817.0,3062656.0,223.0,53.0,132.0,75.0,1070096,1070079,1070087.5
2,3,1000020001,1,2,1,NaN,DEPT OF GENERAL SERVI,Y7,4,709.0,...,6859012.0,1228817.0,3062656.0,223.0,53.0,132.0,75.0,1069397,1069367,1069382.0
3,4,1000020023,1,2,23,NaN,DEPARTMENT OF BUSINES,T2,4,793.0,...,6859012.0,1228817.0,3062656.0,223.0,53.0,132.0,75.0,1068747,1068481,1068614.0
4,5,1000030001,1,3,1,NaN,PARKS AND RECREATION,Q1,4,323.0,...,6859012.0,1228817.0,3062656.0,223.0,53.0,132.0,75.0,1070704,1070701,1070702.5


In [34]:
mergeddata['weightedscore'].sort_values(ascending=False).head(10)

632815     1070994.0
565391     1070993.0
585117     1070992.0
1067359    1070991.0
585438     1070990.0
85885      1070989.0
585119     1070988.0
920627     1070987.0
935157     1070986.0
565397     1070985.0
Name: weightedscore, dtype: float64

Filter the weild records

In [69]:
weird = mergeddata[mergeddata['weightedscore']>=1070985]
weird[['RECORD']]

,RECORD
85885,85886
565391,565392
565397,565398
585117,585118
585119,585120
585438,585439
632815,632816
920627,920628
935157,935158
1067359,1067360


Check whether the FULLVAL/AVLAND/AVTOT/ZIP/STORIES value were NAN in the original dataset



In [27]:
notfill =  pd.read_csv('NY property data.csv')

In [73]:
fill = pd.read_csv('NY property data_filled_Revised.csv')

In [72]:
top10_origin=notfill[value['RECORD'].isin(weird['RECORD'])]
pd.DataFrame(top10_origin).to_csv('top10_origin.csv')

In [74]:
top10_filled=fill[value['RECORD'].isin(weird['RECORD'])]
pd.DataFrame(top10_filled).to_csv('top10_filled.csv')

In [29]:
notfill[['FULLVAL','AVLAND','AVTOT','ZIP','STORIES']][value['RECORD']==684704 ]

,FULLVAL,AVLAND,AVTOT,ZIP,STORIES
684703,0.0,0.0,0.0,NaN,NaN


In [30]:
notfill[['FULLVAL','AVLAND','AVTOT','ZIP','STORIES']][value['RECORD']==776306 ]

,FULLVAL,AVLAND,AVTOT,ZIP,STORIES
776305,0.0,0.0,0.0,NaN,1.0


In [31]:
notfill[['FULLVAL','AVLAND','AVTOT','ZIP','STORIES']][value['RECORD']==821853 ]

,FULLVAL,AVLAND,AVTOT,ZIP,STORIES
821852,0.0,0.0,0.0,NaN,NaN
